In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
pd.options.display.max_columns= 100

In [3]:
### Set working directory
os.chdir('E:/CR2/Repos/TNC-Demand-Model-Southeast/')

In [4]:
### Import wac 2019 data from LEHD lodes
wac = pd.read_csv('Raw_Inputs/LEHD/ky_wac_S000_JT00_2019.csv')
wac.head()

,w_geocode,C000,CA01,CA02,CA03,CE01,CE02,CE03,CNS01,CNS02,CNS03,CNS04,CNS05,CNS06,CNS07,CNS08,CNS09,CNS10,CNS11,CNS12,CNS13,CNS14,CNS15,CNS16,CNS17,CNS18,CNS19,CNS20,CR01,CR02,CR03,CR04,CR05,CR07,CT01,CT02,CD01,CD02,CD03,CD04,CS01,CS02,CFA01,CFA02,CFA03,CFA04,CFA05,CFS01,CFS02,CFS03,CFS04,CFS05,createdate
0,210019701001018,4,2,2,0,3,1,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,4,0,1,1,0,0,0,4,0,0,0,0,0,0,0,0,0,0,20230321
1,210019701001042,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,20230321
2,210019701002012,15,3,9,3,0,6,9,0,0,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,0,0,0,0,0,15,0,2,5,5,0,15,0,0,0,0,0,0,0,0,0,0,0,20230321
3,210019701002014,2,0,2,0,0,2,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,0,0,1,1,0,2,0,0,0,0,0,0,0,0,0,0,0,20230321
4,210019702001002,2,0,1,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,2,0,0,0,0,0,2,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,20230321


In [5]:
# import wac xwalk data from LEHD lodes

In [6]:
geo_xwalk = pd.read_csv('Raw_Inputs/LEHD/ky_xwalk.csv', low_memory=False)

In [7]:
wac_geo = wac.merge(geo_xwalk, how="left", left_on='w_geocode', right_on='tabblk2020')

In [8]:
wac_geo_grouped = wac_geo.groupby(by = 'trct', as_index = False).sum()

In [9]:
# cleaning and processing

In [10]:
wac_geo_grouped['FOOD_EMP'] = wac_geo_grouped['CNS18'] 

In [11]:
wac_geo_grouped['RETAIL_EMP'] = wac_geo_grouped['CNS07'] 

In [12]:
wac_geo_grouped['OTHER_EMP'] = wac_geo_grouped['CNS01'] + wac_geo_grouped['CNS02'] + wac_geo_grouped['CNS03'] + wac_geo_grouped['CNS04'] + wac_geo_grouped['CNS05'] + wac_geo_grouped['CNS06'] + wac_geo_grouped['CNS08'] + wac_geo_grouped['CNS09'] + wac_geo_grouped['CNS10'] + wac_geo_grouped['CNS11'] + wac_geo_grouped['CNS12'] + wac_geo_grouped['CNS13'] + wac_geo_grouped['CNS14'] + wac_geo_grouped['CNS15'] + wac_geo_grouped['CNS16'] + wac_geo_grouped['CNS17'] + wac_geo_grouped['CNS19'] + wac_geo_grouped['CNS20']

In [13]:
wac_geo_grouped.head()

,trct,w_geocode,C000,CA01,CA02,CA03,CE01,CE02,CE03,CNS01,CNS02,CNS03,CNS04,CNS05,CNS06,CNS07,CNS08,CNS09,CNS10,CNS11,CNS12,CNS13,CNS14,CNS15,CNS16,CNS17,CNS18,CNS19,CNS20,CR01,CR02,CR03,CR04,CR05,CR07,CT01,CT02,CD01,CD02,CD03,CD04,CS01,CS02,CFA01,CFA02,CFA03,CFA04,CFA05,CFS01,CFS02,CFS03,CFS04,CFS05,createdate_x,tabblk2020,st,stusps,stname,cty,ctyname,trctname,bgrp,bgrpname,cbsa,cbsaname,zcta,zctaname,stplc,stplcname,ctycsub,ctycsubname,stcd116,stcd116name,stsldl,stsldlname,stsldu,stslduname,stschool,stschoolname,stsecon,stseconname,trib,tribname,tsub,tsubname,stanrc,stanrcname,necta,nectaname,mil,milname,stwib,stwibname,blklatdd,blklondd,createdate_y,FOOD_EMP,RETAIL_EMP,OTHER_EMP
0,21001970100,840078804006086,22,5,14,3,4,9,9,0,0,0,15,0,2,5,0,0,0,0,0,0,0,0,0,0,0,0,0,22,0,0,0,0,0,22,0,3,8,6,0,17,5,0,0,0,0,0,0,0,0,0,0,80921284,840078804006086,84,KYKYKYKY,KentuckyKentuckyKentuckyKentucky,84004,"Adair County, KYAdair County, KYAdair County, ...","9701 (Adair, KY)9701 (Adair, KY)9701 (Adair, K...",840078804006,"1 (Tract 9701, Adair, KY)1 (Tract 9701, Adair,...",399996,0,171012,171012.0,39999996,0,8400762496,"Casey Creek CCD (Adair, KY)Casey Creek CCD (Ad...",8404,KY-01KY-01KY-01KY-01,84204,"State House District 51, KYState House Distric...",84064,"State Senate District 16, KYState Senate Distr...",8400120,"Adair County School District, KYAdair County S...",39999996,0,399996,0.0,39999996,0.0,39999996,0.0,399996,0.0,0.0,0,2121000H2121000H2121000H2121000H,H Cumberlands WIBH Cumberlands WIBH Cumberland...,148.962489,-340.665383,80924064,0,5,17
1,21001970200,3360315232016533,148,35,77,36,22,72,54,2,0,0,19,23,13,2,12,0,26,0,3,0,8,2,35,0,2,1,0,138,8,0,1,0,1,141,7,15,33,36,29,72,76,0,0,0,0,0,0,0,0,0,0,323685136,3360315232016533,336,KYKYKYKYKYKYKYKYKYKYKYKYKYKYKYKY,KentuckyKentuckyKentuckyKentuckyKentuckyKentuc...,336016,"Adair County, KYAdair County, KYAdair County, ...","9702 (Adair, KY)9702 (Adair, KY)9702 (Adair, K...",3360315232016,"1 (Tract 9702, Adair, KY)1 (Tract 9702, Adair,...",1599984,0,683663,683663.0,159999984,0,33603048832,"Cane Valley CCD (Adair, KY)Cane Valley CCD (Ad...",33616,KY-01KY-01KY-01KY-01KY-01KY-01KY-01KY-01KY-01K...,336816,"State House District 51, KYState House Distric...",336256,"State Senate District 16, KYState Senate Distr...",33600480,"Adair County School District, KYAdair County S...",159999984,0,1599984,0.0,159999984,0.0,159999984,0.0,1599984,0.0,0.0,0,2121000H2121000H2121000H2121000H2121000H212100...,H Cumberlands WIBH Cumberlands WIBH Cumberland...,594.698995,-1365.167390,323696256,2,2,144
2,21001970300,3360315248032327,120,26,66,28,43,59,18,0,0,0,7,37,0,17,8,21,1,0,0,0,0,0,5,0,3,21,0,118,2,0,0,0,0,120,0,13,34,24,23,73,47,0,0,0,0,0,0,0,0,0,0,323685136,3360315248032327,336,KYKYKYKYKYKYKYKYKYKYKYKYKYKYKYKY,KentuckyKentuckyKentuckyKentuckyKentuckyKentuc...,336016,"Adair County, KYAdair County, KYAdair County, ...","9703 (Adair, KY)9703 (Adair, KY)9703 (Adair, K...",3360315248032,"1 (Tract 9703, Adair, KY)1 (Tract 9703, Adair,...",1599984,0,682904,682904.0,159999984,0,33603098880,"White Oak CCD (Adair, KY)White Oak CCD (Adair,...",33616,KY-01KY-01KY-01KY-01KY-01KY-01KY-01KY-01KY-01K...,336816,"State House District 51, KYState House Distric...",336256,"State Senate District 16, KYState Senate Distr...",33600480,"Adair County School District, KYAdair County S...",159999984,0,1599984,0.0,159999984,0.0,159999984,0.0,1599984,0.0,0.0,0,2121000H2121000H2121000H2121000H2121000H212100...,H Cumberlands WIBH Cumberlands WIBH Cumberland...,594.015527,-1362.645257,323696256,3,17,100
3,21001970401,9660906384551345,2495,645,1202,648,679,1136,680,3,15,0,54,210,31,592,22,11,143,37,30,23,25,426,355,0,300,21,197,2350,95,8,10,0,32,2459,36,219,620,609,402,1090,1405,0,0,0,0,0,0,0,0,0,0,930594766,9660906384551345,966,KYKYKYKYKYKYKYKYKYKYKYKYKYKYKYKYKYKYKYKYKYKYKY...,KentuckyKentuckyKentuckyKentuckyKentuckyKentuc...,966046,"Adair County, KYAdair County, KYAdair County, ...","9704.01 (Adair, KY)9704.01 (Adair, KY)9704.01 ...",966090

In [31]:
# import shapefile from QGIS (after calculating land area using the shapefile) 

In [14]:
land_area = pd.read_csv('Raw_Inputs/LEHD/census_landarea.csv')

In [15]:
land_area.rename(columns={'GEOID': 'trct'}, inplace=True)
land_area.rename(columns={'LAND_AREA': 'land_area'}, inplace=True)

In [16]:
land_area.tail()

,STATEFP,COUNTYFP,TRACTCE,trct,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,land_area
1301,21,67,900,21067000900,9.00,Census Tract 9,G5020,S,2200382,0,38.043866,-84.513457,0.8496
1302,21,73,70401,21073070401,704.01,Census Tract 704.01,G5020,S,11695102,226634,38.195617,-84.905688,4.6030
1303,21,15,70311,21015070311,703.11,Census Tract 703.11,G5020,S,17799876,495395,39.059426,-84.629649,7.0638
1304,21,15,70314,21015070314,703.14,Census Tract 703.14,G5020,S,3387058,0,38.998637,-84.675483,1.3077
1305,21,25,920200,21025920200,9202.00,Census Tract 9202,G5020,S,187276744,1421183,37.562053,-83.513985,72.8563


In [17]:
wac_geo_grouped.head()

,trct,w_geocode,C000,CA01,CA02,CA03,CE01,CE02,CE03,CNS01,CNS02,CNS03,CNS04,CNS05,CNS06,CNS07,CNS08,CNS09,CNS10,CNS11,CNS12,CNS13,CNS14,CNS15,CNS16,CNS17,CNS18,CNS19,CNS20,CR01,CR02,CR03,CR04,CR05,CR07,CT01,CT02,CD01,CD02,CD03,CD04,CS01,CS02,CFA01,CFA02,CFA03,CFA04,CFA05,CFS01,CFS02,CFS03,CFS04,CFS05,createdate_x,tabblk2020,st,stusps,stname,cty,ctyname,trctname,bgrp,bgrpname,cbsa,cbsaname,zcta,zctaname,stplc,stplcname,ctycsub,ctycsubname,stcd116,stcd116name,stsldl,stsldlname,stsldu,stslduname,stschool,stschoolname,stsecon,stseconname,trib,tribname,tsub,tsubname,stanrc,stanrcname,necta,nectaname,mil,milname,stwib,stwibname,blklatdd,blklondd,createdate_y,FOOD_EMP,RETAIL_EMP,OTHER_EMP
0,21001970100,840078804006086,22,5,14,3,4,9,9,0,0,0,15,0,2,5,0,0,0,0,0,0,0,0,0,0,0,0,0,22,0,0,0,0,0,22,0,3,8,6,0,17,5,0,0,0,0,0,0,0,0,0,0,80921284,840078804006086,84,KYKYKYKY,KentuckyKentuckyKentuckyKentucky,84004,"Adair County, KYAdair County, KYAdair County, ...","9701 (Adair, KY)9701 (Adair, KY)9701 (Adair, K...",840078804006,"1 (Tract 9701, Adair, KY)1 (Tract 9701, Adair,...",399996,0,171012,171012.0,39999996,0,8400762496,"Casey Creek CCD (Adair, KY)Casey Creek CCD (Ad...",8404,KY-01KY-01KY-01KY-01,84204,"State House District 51, KYState House Distric...",84064,"State Senate District 16, KYState Senate Distr...",8400120,"Adair County School District, KYAdair County S...",39999996,0,399996,0.0,39999996,0.0,39999996,0.0,399996,0.0,0.0,0,2121000H2121000H2121000H2121000H,H Cumberlands WIBH Cumberlands WIBH Cumberland...,148.962489,-340.665383,80924064,0,5,17
1,21001970200,3360315232016533,148,35,77,36,22,72,54,2,0,0,19,23,13,2,12,0,26,0,3,0,8,2,35,0,2,1,0,138,8,0,1,0,1,141,7,15,33,36,29,72,76,0,0,0,0,0,0,0,0,0,0,323685136,3360315232016533,336,KYKYKYKYKYKYKYKYKYKYKYKYKYKYKYKY,KentuckyKentuckyKentuckyKentuckyKentuckyKentuc...,336016,"Adair County, KYAdair County, KYAdair County, ...","9702 (Adair, KY)9702 (Adair, KY)9702 (Adair, K...",3360315232016,"1 (Tract 9702, Adair, KY)1 (Tract 9702, Adair,...",1599984,0,683663,683663.0,159999984,0,33603048832,"Cane Valley CCD (Adair, KY)Cane Valley CCD (Ad...",33616,KY-01KY-01KY-01KY-01KY-01KY-01KY-01KY-01KY-01K...,336816,"State House District 51, KYState House Distric...",336256,"State Senate District 16, KYState Senate Distr...",33600480,"Adair County School District, KYAdair County S...",159999984,0,1599984,0.0,159999984,0.0,159999984,0.0,1599984,0.0,0.0,0,2121000H2121000H2121000H2121000H2121000H212100...,H Cumberlands WIBH Cumberlands WIBH Cumberland...,594.698995,-1365.167390,323696256,2,2,144
2,21001970300,3360315248032327,120,26,66,28,43,59,18,0,0,0,7,37,0,17,8,21,1,0,0,0,0,0,5,0,3,21,0,118,2,0,0,0,0,120,0,13,34,24,23,73,47,0,0,0,0,0,0,0,0,0,0,323685136,3360315248032327,336,KYKYKYKYKYKYKYKYKYKYKYKYKYKYKYKY,KentuckyKentuckyKentuckyKentuckyKentuckyKentuc...,336016,"Adair County, KYAdair County, KYAdair County, ...","9703 (Adair, KY)9703 (Adair, KY)9703 (Adair, K...",3360315248032,"1 (Tract 9703, Adair, KY)1 (Tract 9703, Adair,...",1599984,0,682904,682904.0,159999984,0,33603098880,"White Oak CCD (Adair, KY)White Oak CCD (Adair,...",33616,KY-01KY-01KY-01KY-01KY-01KY-01KY-01KY-01KY-01K...,336816,"State House District 51, KYState House Distric...",336256,"State Senate District 16, KYState Senate Distr...",33600480,"Adair County School District, KYAdair County S...",159999984,0,1599984,0.0,159999984,0.0,159999984,0.0,1599984,0.0,0.0,0,2121000H2121000H2121000H2121000H2121000H212100...,H Cumberlands WIBH Cumberlands WIBH Cumberland...,594.015527,-1362.645257,323696256,3,17,100
3,21001970401,9660906384551345,2495,645,1202,648,679,1136,680,3,15,0,54,210,31,592,22,11,143,37,30,23,25,426,355,0,300,21,197,2350,95,8,10,0,32,2459,36,219,620,609,402,1090,1405,0,0,0,0,0,0,0,0,0,0,930594766,9660906384551345,966,KYKYKYKYKYKYKYKYKYKYKYKYKYKYKYKYKYKYKYKYKYKYKY...,KentuckyKentuckyKentuckyKentuckyKentuckyKentuc...,966046,"Adair County, KYAdair County, KYAdair County, ...","9704.01 (Adair, KY)9704.01 (Adair, KY)9704.01 ...",966090

In [18]:
# merged 

In [19]:
wac_geo_grouped = pd.merge(wac_geo_grouped, land_area, on='trct', how='inner')

In [20]:
wac_geo_grouped['TOTAL_EMP_DEN'] = (wac_geo_grouped['FOOD_EMP'] + wac_geo_grouped['RETAIL_EMP'] + wac_geo_grouped['OTHER_EMP']) / wac_geo_grouped['land_area']

In [21]:
wac_geo_grouped = wac_geo_grouped[['trct', 'FOOD_EMP', 'RETAIL_EMP', 'OTHER_EMP', 'TOTAL_EMP_DEN']]

In [22]:
den_sum = wac_geo_grouped['TOTAL_EMP_DEN'].sum()
den_sum

990275.0737692777

In [23]:
wac_geo_grouped.head()

,trct,FOOD_EMP,RETAIL_EMP,OTHER_EMP,TOTAL_EMP_DEN
0,21001970100,0,5,17,0.311002
1,21001970200,2,2,144,3.430007
2,21001970300,3,17,100,1.615294
3,21001970401,300,592,1603,60.917548
4,21001970402,100,115,1328,59.163884


In [24]:
wac_geo_grouped.to_csv('Cleaned_Inputs/Intermediate/wac.csv')